Create video with plot visualization of force trajectory over time.
1. Video of top and side views with tracking of the pendulum rod tip.
2. Contact point of the stem along the rod- in side view.
3. visualization of force trajectory over time (with deflection, angle, and contact as well?).
4. maybe add an arrow of the Fxy direction in the top view video?

In [2]:
# import libraries

# generic
import sys
import numpy as np
import os,glob # for listing files in folder
import re # regular expressions
import pandas as pd
import scipy
import seaborn
import math as m
import cv2

from scipy.signal import savgol_filter
from scipy.spatial import distance as sci_distance
from scipy.stats import kruskal
import importlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.lines as mlines
import random
import itertools
from tqdm import tqdm
import pickle as pkl
import datetime
now = datetime.datetime.now()

# custom
import article0_funcs_clean # writch to updated exp2 functions
# sys.path.append('..')
import useful_functions as uf 

#general parameters
cmap = plt.get_cmap('viridis')
fs = 16 # standard font size for plots


print(os.getcwd())

importlib.reload(uf)
importlib.reload(article0_funcs_clean)

save_folder = r'C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Images'


hi
c:\Users\Amir\Documents\PHD\Python\GitHub\Amir_Repositories\Repo_article0
hi


In [ ]:
# import data 
root_path = r'C:\Users\Amir\Documents\PHD\Experiments\Force Measurements'
basepath = root_path+r'\Exp2_Pendulum' # pendulum exp folder
# excel_csv= r'\Exp2_supplementary_measurements_events.csv'
data_excel = r'\Exp2_supplementary_measurements_events_xl.xlsx'

# straw exp data
# data_panda = pd.read_csv(basepath+excel_csv)
data_panda = pd.read_excel(basepath+data_excel)


# import tracked data, initiate variables
track_sup_path = r'\track_logs'
track_folder_path = basepath+track_sup_path

track_contact_path = r'\contact_track_logs'
contact_folder_path = basepath+track_contact_path

# track_near_sup_path = r'\twine_init_logs'
# track_near_sup_folder_path = basepath+track_near_sup_path

# h5_path = r'\Measurements\root_stem_results'
# h5_folder_path = basepath+h5_path

# E_path = r'\Young_moduli'
# E_folder_path = basepath+E_path

N_track = len(os.listdir(track_folder_path)) # get number of files in track folder
N_contact=len(os.listdir(contact_folder_path)) # get number of files in contact folder
N_tot = len(data_panda) # get number of lines in excel
# N_E = len(os.listdir(E_folder_path))


#%% remove problematic events from raw data
delete_rows = [] # save rows to delete
problem_exp = [] # exp_num of problem events

for i in range(N_tot): # remove problem events and non-Helda events
    if data_panda.at[i,'problem']!='na' or data_panda.at[i,'Bean_Strain']!='Helda':
        delete_rows.append(i)
        problem_exp.append(data_panda.at[i,'Exp_num'])
N = N_tot - len(delete_rows) # modify num of rows

data_panda = data_panda.drop(data_panda.index[delete_rows]) # remove prob. events
data_panda = data_panda.reset_index() # redo index
#%% get misc. file lists

# get track files for support bottom coordinates
remove_chars = re.compile('[,_\.!?]') # what to remove from strings
track_dict = {} # save support track in dictionary by exp and events
i=0 # start with first track file
for file in glob.glob(os.path.join(track_folder_path, '*.txt')): # for each event:
    exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
    event = int(re.findall('[0-9]',re.findall('_\d{1}\D',file)[0].replace('_',''))[0]) # find event number
    viewt = re.findall('(side{1}|top{1})',file)[0] #.replace('_','')
    track_dict[(exp,event,viewt)] = [file] # add new exp
    i+=1

# get track files for stem-support contact coordinates
contact_dict = {} # save support contact track in dictionary by exp and events
i=0 # start with first contact file
for file in glob.glob(os.path.join(contact_folder_path, '*.txt')): # for each event:
    exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
    event = int(re.findall('_[0-9]_',file)[0].replace('_','')) # find event number
    contact_dict[(exp,event)] = [file] # add new exp
    i+=1

# get track files for 2 stem positions on either side of support
# near_sup_track_dict = {}
# i=0 # start with first stem_near_sup file
# for file in glob.glob(os.path.join(track_near_sup_folder_path, '*.txt')): # for each event:
#     exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
#     event = int(re.findall('_[0-9]_',file)[0].replace('_','')) # find event number
#     near_sup_track_dict[(exp,event)] = [file] # add new exp
#     i+=1

# get h5 files for stem near support
# h5_dict = {}
# i=0 # start with first h5 file
# for file in glob.glob(os.path.join(h5_folder_path, '*.h5')): # for each event:
#     exp = int(re.findall('interekt_\d{2,3}_',file)[0].split('_')[1]) # find exp number (3-4 digits)
#     event = int(re.findall('e_[0-9]_',file)[0].split('_')[1]) # find event number
#     start_frame = int(re.findall('_\d{2,5}-\d{2,5}',file)[0].replace('_','').split('-')[0]) # find start frame
#     h5_dict[(exp,event,start_frame)] = [file] # add new exp
#     i+=1

# get Young modulus files
# E_dict = {}
# for file in glob.glob(os.path.join(E_folder_path, '*.csv')):
#     exp = int(re.findall('\d{2,3}',file)[0])
#     E_dict[exp]=pd.read_csv(file,header=None)

#%%

In [ ]:
# clear plants and events
plants = []
events = []
#%% populate plant and event instances
i = 0
N = len(data_panda)
for i in tqdm(range(N)): # N
    try:
        exp = int(re.findall('\d{3,4}',data_panda.at[i,'Exp_num'])[0]) # exp num
        view = data_panda.at[i,'View']  # side of top view
        if view == 'top':
            plants[-1].pix2cm_t = float(data_panda.at[i,'Top_pix2cm'])

        if i==0 or exp!=plants[-1].exp_num: # append new plant with data from pandas
            #basic data
            plants.append(article0_funcs_clean.Plant(data_panda,basepath,i,exp))
            # view dependent data
            plants[-1].view_data(data_panda,i)
            # circumnutation data
            plants[-1].cn_data(data_panda,i)
            #Youngs modulus by segment with avg
            # plants[-1].getE(E_dict)


        event =  int(re.findall('_[0-9]',data_panda.at[i,
        'Exp_num'])[0].replace('_','')) # get event number


        # if this is the 1st event or the previous event_num is different from the current one:
        # add new event to list
        if len(events)==0 or events[-1].event_num != event or \
            events[-1].p.exp_num != exp:
            events.append(article0_funcs_clean.Event(plants[-1],data_panda,i))
        events[-1].event_num = event

        # view dependent data
        events[-1].view_data(data_panda,i,view)

        # get automated extraction of twine(decision) time
        # events[-1].get_twine_time(exp,event,view,
        #                   h5_dict,near_sup_track_dict,50,track_dict,to_plot=0)


        # get track data, select decision period data, pix2cm,
        events[-1].event_base_calcs(view,track_dict,contact_dict)
        # calc
        events[-1].event_calc_variables(view)
        # print(f'i={i}, exp number {exp}, {event}') # print progress
    except Exception as e:
        print(f'Error at i={i}, exp number {exp}, {event}')
        print(e)
        
        continue


In [ ]:
# check new measurements
for i in range(len(events)):
    events[i].support_base_z_pos_pix_new

In [1]:
# define create video of side and top view in split screen for a specific event
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


def parallel_video(images1, coords1, images2, coords2, fps=2, fixed_height=300):
    """
    Display two sets of images side by side with markers in a resizable window.
    
    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final window.
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2):
        raise ValueError("The number of images and coordinates must match for both sets.")
    if len(images1) != len(images2):
        raise ValueError("Both sets of images must have the same length.")
    
    delay = 1 / fps  # Delay in seconds between frames
    
    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    for idx, (img1, coord1, img2, coord2) in enumerate(zip(images1, coords1, images2, coords2)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)
        
        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
        
        # Draw markers on each image
        if coord1:
            cv2.circle(img1, (int(coord1[0]), int(coord1[1])), radius=10, color=(0, 165, 255), thickness=-1)
        if coord2:
            cv2.circle(img1, (int(coord2[0]), int(coord2[1])), radius=10, color=(0, 165, 255), thickness=-1)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.drawMarker(img1, (int(prev_coord1[0]), int(prev_coord1[1])), color=(0, 255, 0), 
                   markerType=cv2.MARKER_CROSS, thickness=2, markerSize=15)
        for prev_coord2 in coords2[:idx]:
            cv2.drawMarker(img2, (int(prev_coord2[0]), int(prev_coord2[1])), color=(0, 255, 0), 
                   markerType=cv2.MARKER_TILTED_CROSS, thickness=2, markerSize=15)
        # Make both images the same height for horizontal concatenation
        height = max(img1.shape[0], img2.shape[0])
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))
        
        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])
        
        # Scale the combined image to have a fixed height
        scale_ratio = fixed_height / combined_image.shape[0]
        scaled_width = int(combined_image.shape[1] * scale_ratio)
        resized_image = cv2.resize(combined_image, (scaled_width, fixed_height))
        
        # Set the window size to the resized image dimensions
        cv2.resizeWindow(window_name, resized_image.shape[1], resized_image.shape[0])
        
        # Display the resized concatenated image in the window
       
        # Calculate the position to center the window
        window_x = 1400
        window_y = 50
        
        # Move the window to the center of the screen
        cv2.moveWindow(window_name, window_x, window_y)
        
        cv2.imshow(window_name, resized_image)
        
        # Wait for the specified delay or until a key is pressed
        if idx == len(images1) - 1:
            # Keep the last image displayed until the user closes the window
            print("Final image displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))  # Convert delay to milliseconds
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()


def display_images_with_graph(images1, coords1, images2, coords2, graph_data, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"):
    """
    Display two sets of images side by side with a live-updating graph below them, 
    and optionally save the sequence as a video.

    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        graph_data (list): List of (x, y) tuples for graph plotting.
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final display window.
        save_video (bool): Whether to save the sequence as a video.
        output_path (str): Path to save the video (if enabled).
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2) or len(images1) != len(graph_data):
        raise ValueError("The number of images, coordinates, and graph data points must match.")
    
    delay = 1 / fps  # Delay in seconds between frames
    
    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    for idx, (img1, coord1, img2, coord2, (x_vals, y_vals)) in enumerate(zip(images1, coords1, images2, coords2, graph_data)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)
        
        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
        
        # Draw markers on each image
        if coord1:
            cv2.drawMarker(img1, (int(coord1[0]), int(coord1[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_CROSS, thickness=3, markerSize=50)
        if coord2:
            cv2.drawMarker(img2, (int(coord2[0]), int(coord2[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_TILTED_CROSS, thickness=3, markerSize=50)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.drawMarker(img1, (int(prev_coord1[0]), int(prev_coord1[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_CROSS, thickness=2, markerSize=40)
        for prev_coord2 in coords2[:idx]:
            cv2.drawMarker(img2, (int(prev_coord2[0]), int(prev_coord2[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_TILTED_CROSS, thickness=2, markerSize=40)
            
        # Resize images to the fixed height
        height = fixed_height
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))
        
        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])
        
        # Create the graph as an image
        fig, ax = plt.subplots(figsize=(4, 3))
        max_x = max(graph_data, key=lambda x: x[0])[0]
        max_y = max(graph_data, key=lambda x: x[1])[1]
        ax.set_xlim([0, max_x * 1.1])
        ax.set_ylim([0, max_y * 1.1])
        ax.plot(x_vals, y_vals, marker="o", color="blue", linewidth=2)
        # for prev_xy in graph_data[:idx]:
        for prev_x_vals, prev_y_vals in graph_data[:idx]:
            ax.plot(prev_x_vals, prev_y_vals,  marker="o", color="black", linewidth=2)
        ax.set_xlabel("t(sec)")
        ax.set_ylabel("F(mN)")
        ax.grid(True)
        canvas = FigureCanvas(fig)
        canvas.draw()
        graph_img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
        graph_img = graph_img.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)
        
        # Resize the graph to match the combined image width
        graph_height = int(combined_image.shape[0] * 0.5)  # Scale graph height relative to images
        graph_img = cv2.resize(graph_img, (combined_image.shape[1], graph_height))
        
        # Concatenate graph below the images
        final_frame = cv2.vconcat([combined_image, graph_img])
        
        if video_writer is None and save_video:
            frame_height, frame_width = final_frame.shape[:2]
            video_writer = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*"mp4v"),
                fps,
                (frame_width, frame_height),
                )

        # Resize window to match final frame dimensions
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])
        
        # Calculate the position to center the window
        window_x = 100
        window_y = 50
        
        # Move the window to the center of the screen
        cv2.moveWindow(window_name, window_x, window_y)
        
        # Display the final frame
        cv2.imshow(window_name, final_frame)
        
        # Write to video if saving
        if save_video and video_writer:
            video_writer.write(cv2.cvtColor(final_frame, cv2.COLOR_RGB2BGR))
        
        # Wait for the specified delay or until a key is pressed
        if idx == len(images1) - 1:
            # Keep the last frame displayed until the user closes the window
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))  # Convert delay to milliseconds
    
    # Release the video writer
    if save_video and video_writer:
        video_writer.release()
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()

def display_images_with_graph2(
    images1, coords1, images2, coords2, graph_data, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"
):
    """
    Display two sets of images side by side with a live-updating graph below them,
    and optionally save the sequence as a video.

    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        graph_data (list): List of (x, y) tuples for graph plotting.
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final display window.
        save_video (bool): Whether to save the sequence as a video.
        output_path (str): Path to save the video (if enabled).
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2) or len(images1) != len(graph_data):
        raise ValueError("The number of images, coordinates, and graph data points must match.")

    delay = 1 / fps  # Delay in seconds between frames

    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    # Initialize video writer if saving the video
    video_writer = None
    frame_width = 2 * fixed_height  # Width proportional to height
    frame_height = int(fixed_height + 200)  # Extra height for graph
    if save_video:
        video_writer = cv2.VideoWriter(
            output_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (frame_width, frame_height),
        )

    for idx, (img1, coord1, img2, coord2, (x_vals, y_vals)) in enumerate(zip(images1, coords1, images2, coords2, graph_data)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)

        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        # Draw markers on each image
        if coord1:
            cv2.circle(img1, (int(coord1[0]), int(coord1[1])), radius=15, color=(0, 165, 255), thickness=-1)
        if coord2:
            cv2.circle(img2, (int(coord2[0]), int(coord2[1])), radius=15, color=(0, 165, 255), thickness=-1)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.circle(img1, (int(prev_coord1[0]), int(prev_coord1[1])), radius=12, color=(255, 0, 0), thickness=-1)
        for prev_coord2 in coords2[:idx]:
            cv2.circle(img2, (int(prev_coord2[0]), int(prev_coord2[1])), radius=12, color=(255, 0, 0), thickness=-1)
            
        # Resize images to the fixed height
        height = fixed_height
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))

        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])
        # Create the graph as an image
        fig, ax = plt.subplots(figsize=(8, 4))  # Adjust width and height for a better aspect ratio
        plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.2)  # Adjust margins
        ax.plot(x_vals, y_vals, marker="o", color='orange', linewidth=1.5)
        for prev_x_vals, prev_y_vals in graph_data[:idx]:
            ax.plot(prev_x_vals, prev_y_vals, marker="o", color='blue', linewidth=1.5)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Force (mN)", labelpad=15, fontsize=12)
        max_x = max(graph_data, key=lambda x: x[0])[0]
        max_y = max(graph_data, key=lambda x: x[1])[1]
        ax.set_xlim([0, max_x * 1.1])
        ax.set_ylim([0, max_y * 1.1])
        ax.grid(True)
        canvas = FigureCanvas(fig)
        canvas.draw()
        graph_img = np.frombuffer(canvas.tostring_rgb(), dtype="uint8")
        graph_img = graph_img.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        # Resize the graph while retaining proportions
        graph_height, graph_width = graph_img.shape[:2]
        scale_ratio = combined_image.shape[1] / graph_width
        new_graph_width = int(graph_width * scale_ratio)
        new_graph_height = int(graph_height * scale_ratio)
        graph_img = cv2.resize(graph_img, (new_graph_width, new_graph_height))

        # Resize the graph to match the width of the combined image
        # graph_img = cv2.resize(graph_img, (combined_image.shape[1], 200))
        # Convert graph image to RGB
        graph_img = cv2.cvtColor(graph_img, cv2.COLOR_BGR2RGB)
        # Concatenate graph below the images
        final_frame = cv2.vconcat([combined_image, graph_img])
        
        # Resize the graph while retaining proportions
        # graph_height, graph_width = graph_img.shape[:2]
        # scale_ratio = combined_image.shape[1] / graph_width
        # new_graph_width = int(graph_width * scale_ratio)
        # new_graph_height = int(graph_height * scale_ratio)
        # graph_img = cv2.resize(graph_img, (new_graph_width, new_graph_height))

        # Concatenate graph below the images
        final_frame = cv2.vconcat([combined_image, graph_img])

        # Resize window to match final frame dimensions
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])

        # Display the final frame
        cv2.imshow(window_name, final_frame)

        # Initialize video writer if not already initialized
        if save_video and video_writer is None:
            frame_width, frame_height = final_frame.shape[1], final_frame.shape[0]
            video_writer = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*"mp4v"),
                fps,
                (frame_width, frame_height),
            )

        # Now write the frame
        if save_video:
            video_writer.write(final_frame)


        if idx == len(images1) - 1:
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))

    if save_video and video_writer:
        video_writer.release()

    cv2.destroyAllWindows()

    #####################
    # Save the last frame of the video separately
    # last_frame_path = os.path.join(save_folder, "last_frame_video.jpg")
    # cv2.imwrite(last_frame_path, resized_frame_for_video)

    # # Save the last frame as shown in the cv2 window separately
    # last_frame_window_path = os.path.join(save_folder, "last_frame_window.jpg")
    # cv2.imwrite(last_frame_window_path, final_frame)

    # # Combine the two images side by side
    # last_frame_video = cv2.imread(last_frame_path)
    # last_frame_window = cv2.imread(last_frame_window_path)

    # # Ensure both images have the same height for concatenation
    # height = max(last_frame_video.shape[0], last_frame_window.shape[0])
    # last_frame_video = cv2.resize(last_frame_video, (int(last_frame_video.shape[1] * height / last_frame_video.shape[0]), height))
    # last_frame_window = cv2.resize(last_frame_window, (int(last_frame_window.shape[1] * height / last_frame_window.shape[0]), height))

    # # Concatenate the images horizontally
    # combined_last_frames = cv2.hconcat([last_frame_video, last_frame_window])

    # # Save the combined image
    # combined_last_frames_path = os.path.join(save_folder, "combined_last_frames.jpg")
    # cv2.imwrite(combined_last_frames_path, combined_last_frames)

    # print(f"Last frame of video saved at: {last_frame_path}")
    # print(f"Last frame from window saved at: {last_frame_window_path}")
    # print(f"Combined last frames saved at: {combined_last_frames_path}")



def display_images_with_graph3(
    images1, coords1, images2, coords2, graph_data, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"
):
    if len(images1) != len(coords1) or len(images2) != len(coords2) or len(images1) != len(graph_data):
        raise ValueError("The number of images, coordinates, and graph data points must match.")

    delay = 1 / fps
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    video_writer = None  # Will be initialized after first frame

    for idx, (img1, coord1, img2, coord2, (x_vals, y_vals)) in enumerate(zip(images1, coords1, images2, coords2, graph_data)):
        # Load images if they are file paths
        img1 = cv2.imread(img1) if isinstance(img1, str) else img1
        img2 = cv2.imread(img2) if isinstance(img2, str) else img2

        # Convert grayscale to BGR
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        # Draw current and past markers
        if coord1:
            cv2.circle(img1, (int(coord1[0]), int(coord1[1])), radius=15, color=(0, 165, 255), thickness=-1)
        if coord2:
            cv2.circle(img2, (int(coord2[0]), int(coord2[1])), radius=15, color=(0, 165, 255), thickness=-1)
        for prev_coord1 in coords1[:idx]:
            cv2.circle(img1, (int(prev_coord1[0]), int(prev_coord1[1])), radius=12, color=(255, 0, 0), thickness=-1)
        for prev_coord2 in coords2[:idx]:
            cv2.circle(img2, (int(prev_coord2[0]), int(prev_coord2[1])), radius=12, color=(255, 0, 0), thickness=-1)

        # Resize images to fixed height
        img1 = cv2.resize(img1, (int(img1.shape[1] * fixed_height / img1.shape[0]), fixed_height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * fixed_height / img2.shape[0]), fixed_height))

        # Combine images horizontally
        combined_image = cv2.hconcat([img1, img2])

        # Create graph using matplotlib
        fig, ax = plt.subplots(figsize=(8, 4))
        plt.subplots_adjust(left=0.2, right=0.9, top=0.9, bottom=0.2)
        ax.plot(x_vals, y_vals, marker="o", color='orange', linewidth=1.5)
        for prev_x_vals, prev_y_vals in graph_data[:idx]:
            ax.plot(prev_x_vals, prev_y_vals, marker="o", color='blue', linewidth=1.5)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Force (mN)", labelpad=15, fontsize=12)
        max_x = max(graph_data, key=lambda x: x[0])[0]
        max_y = max(graph_data, key=lambda x: x[1])[1]
        ax.set_xlim([0, max_x * 1.1])
        ax.set_ylim([0, max_y * 1.1])
        ax.grid(True)

        canvas = FigureCanvas(fig)
        canvas.draw()
        graph_img = np.frombuffer(canvas.tostring_rgb(), dtype="uint8")
        graph_img = graph_img.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        # Resize graph to match image width
        scale_ratio = combined_image.shape[1] / graph_img.shape[1]
        graph_img = cv2.resize(graph_img, (combined_image.shape[1], int(graph_img.shape[0] * scale_ratio)))

        # Convert graph image to BGR for OpenCV
        graph_img = cv2.cvtColor(graph_img, cv2.COLOR_RGB2BGR)

        # Combine image and graph vertically
        final_frame = cv2.vconcat([combined_image, graph_img])

        # Display
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])
        cv2.imshow(window_name, final_frame)

        # Initialize writer on first frame (ensures correct resolution)
        if save_video and video_writer is None:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            frame_height, frame_width = final_frame.shape[:2]
            video_writer = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*"mp4v"),
                fps,
                (frame_width, frame_height),
            )

        # Write to video
        if save_video:
            video_writer.write(final_frame)

        # Wait for next frame
        if idx == len(images1) - 1:
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))

    if save_video and video_writer:
        video_writer.release()

    cv2.destroyAllWindows()


def parallel_video2(
    images, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"
):
    """
    Display and optionally save a video of multiple synchronized views.

    Parameters:
        images (list of lists): List of N lists of image paths/arrays for N views.
        fps (int): Frames per second.
        fixed_height (int): Height for each image.
        save_video (bool): Whether to save the video.
        output_path (str): Path to save the output video.
    """

    delay = 1 / fps
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    num_views = len(images)
    num_frames = len(images[0])
    
    # Ensure all views have the same number of frames
    if not all(len(view) == num_frames for view in images):
        raise ValueError("All views must have the same number of frames.")

    # Determine layout
    if num_views <= 3:
        layout = "horizontal"
    else:
        layout = "grid"
        grid_cols = m.ceil(m.sqrt(num_views))
        grid_rows = m.ceil(num_views / grid_cols)

    video_writer = None

    for idx in range(num_frames):
        resized_images = []

        # Process each view
        for view in images:
            img = view[idx]
            if isinstance(img, str):
                img = cv2.imread(img)
            if len(img.shape) < 3:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

            img = cv2.resize(img, (int(img.shape[1] * fixed_height / img.shape[0]), fixed_height))
            resized_images.append(img)

        # Combine images
        if layout == "horizontal":
            combined_image = cv2.hconcat(resized_images)
        else:
            # Pad the images list to fill the grid
            while len(resized_images) < grid_rows * grid_cols:
                height, width = resized_images[0].shape[:2]
                resized_images.append(np.zeros((height, width, 3), dtype=np.uint8))
            
            rows = []
            for r in range(grid_rows):
                row_imgs = resized_images[r * grid_cols:(r + 1) * grid_cols]
                row = cv2.hconcat(row_imgs)
                rows.append(row)
            combined_image = cv2.vconcat(rows)

        # Resize window and show image
        cv2.resizeWindow(window_name, combined_image.shape[1], combined_image.shape[0])
        cv2.imshow(window_name, combined_image)

        # Initialize video writer if needed
        if save_video and video_writer is None:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            frame_height, frame_width = combined_image.shape[:2]
            video_writer = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*"mp4v"),
                fps,
                (frame_width, frame_height),
            )

        # Save video
        if save_video:
            video_writer.write(combined_image)

        # Wait
        if idx == num_frames - 1:
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))

    if save_video and video_writer:
        video_writer.release()

    cv2.destroyAllWindows()


In [157]:
#%%  Define the experiment and event numbers
# exp_num = 30
# event_num = 1
# find the event corresponding to exp_num and event_num:
i = 70 # 40 (simple) , 20,120 (bump from dist2tip), i=70->event 79_2
event = events[i]
# event.frm0_top = 10 # original

print(event.p.exp_num,event.event_num)
print(event.frm0_top,event.frm_dec_top)

# start video from initial contact frame for both views

# Define paths to the video frames
side_view_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_side\cropped"
top_view_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_top\cropped"
vid_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory"

top_view_files = [os.path.join(top_view_path, f) for f in os.listdir(top_view_path) if os.path.isfile(os.path.join(top_view_path, f))]
side_view_files = [os.path.join(side_view_path, f) for f in os.listdir(side_view_path) if os.path.isfile(os.path.join(side_view_path, f))]                                                                    

# Define the number of frames
fr = 70
fr = min(fr,len(top_view_files))

top_images = top_view_files[event.frm0_top:event.frm_dec_top][:fr]
side_images = side_view_files[event.frm0_side:event.frm_dec_side][:fr]

#################
# Load coordinates from another track file in the folder
track_side_folder_path = r'C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_side\CSRT_079_2\Rois_cropped_'
track_top_folder_path = r'C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_top\CSRT_079_2\Rois_cropped_'

# List all files in the side and top track folders
track_side_path = os.path.join(track_side_folder_path, 'CSRT_079_side_2.txt')
track_top_path = os.path.join(track_top_folder_path, 'CSRT_079_top_2.txt')

# Read the tracked coordinates from the side file 
# xz_track_side = uf.funcget_tracked_data(track_side_path,[0,-1],'top','nikon')
# read the tracked coordinates
xz_track_side = uf.funcget_tracked_data(track_side_path, [0, -1], 'side','pi')
xy_track_top = uf.funcget_tracked_data(track_top_path, [0, -1], 'top','pi')

#################

# top tracked coordiantes
x_coor_top = xy_track_top[0][event.frm0_top:event.frm_dec_top][:fr]
y_coor_top = xy_track_top[1][event.frm0_top:event.frm_dec_top][:fr]
# side tracked coordiantes
x_coor_side = xz_track_side[0][event.frm0_side:event.frm_dec_side][:fr]
z_coor_side = xz_track_side[1][event.frm0_side:event.frm_dec_side][:fr]

side_xz_coor = list(zip(x_coor_side,z_coor_side))
top_xy_coor = list(zip(x_coor_top,y_coor_top))

t = event.timer[event.frm0_top:event.frm_dec_top][:fr]-event.timer[event.frm0_top]
# force data
force_data = list(zip(t,event.F_bean[event.frm0_top:event.frm_dec_top][:fr]))

# parallel_video(top_images, top_xy_coor,side_images, side_xz_coor, fps=2)
display_images_with_graph3(top_images, top_xy_coor,side_images, side_xz_coor, graph_data=force_data , fps=10,
                          fixed_height=500, save_video=True, output_path=vid_path+"\\side_top_crop_f(t)_video.mp4")


#################


59 5
10 175
50
['436.0', ' 2121.0', ' 68.0', ' 83.0', ' C:\\Users\\Amir\\Documents\\PHD\\Experiments\\Force Measurements\\Exp2.0_Methods-paper\\79_2_side\\cropped\\5775_CROPED.JPG', ' path time not found', ' 0', ' Auto\n']
50
['1687.0', ' 1212.0', ' 406.0', ' 394.0', ' C:\\Users\\Amir\\Documents\\PHD\\Experiments\\Force Measurements\\Exp2.0_Methods-paper\\79_2_top\\cropped\\1067_CROPED.JPG', ' path time not found', ' 0', ' Auto\n']


C:\Users\Amir\AppData\Local\Temp\ipykernel_29160\1515899450.py:452: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  graph_img = np.frombuffer(canvas.tostring_rgb(), dtype="uint8")


Final frame displayed. Close the window to end.


In [134]:
# checks
dec_x_track_top = event.x_track_top_pix[event.frm0_top:event.frm_dec_top][:fr]
dec_y_track_top = event.y_track_top_pix[event.frm0_top:event.frm_dec_top][:fr]
top_xy_coor = list(zip(dec_x_track_top,dec_y_track_top))

# side images and tracked coordiantes
side_images = side_view_files[event.frm0_side:event.frm_dec_side][:fr]
dec_x_track_side = event.x_track_side0[event.frm0_side:event.frm_dec_side][:fr]
dec_y_track_side = event.z_track_side0[event.frm0_side:event.frm_dec_side][:fr]
side_xz_coor = list(zip(dec_x_track_side,dec_y_track_side))



print(f"Number of top images: {len(top_images)}")
print(f"Number of side images: {len(side_images)}")
print(f"Number of top x and y coordinates: {len(x_coor_top)}, {len(y_coor_top)}")
print(f"Number of side x and z coordinates: {len(x_coor_side)}, {len(z_coor_side)}")
# Check the length of the coordinates extracted from funcget_tracked_data
print(f"Length of xz_track_side: {len(xz_track_side[0])}, {len(xz_track_side[1])}")
print(f"Length of xy_track_top: {len(xy_track_top[0])}, {len(xy_track_top[1])}")

# Ensure slicing is done correctly
print(f"Event frame range for top: {event.frm0_top} to {event.frm_dec_top}")
print(f"Event frame range for side: {event.frm0_side} to {event.frm_dec_side}")

# Check the slicing operation
print(f"Length of sliced x_coor_top: {len(x_coor_top)}")
print(f"Length of sliced y_coor_top: {len(y_coor_top)}")
print(f"Length of sliced x_coor_side: {len(x_coor_side)}")
print(f"Length of sliced z_coor_side: {len(z_coor_side)}")
print(f"{fr=}")

# Print the first lines in the track top file
# Show 10 box centers from the file
print("Box centers from the file:")
box_centers_file = []
with open(track_top_path, 'r') as file:
    for i in range(10):  # Extract the first 10 box centers
        line = file.readline().strip()
        values = line.split(',')[:4]  # Extract only the first 4 values
        x_left, y_top, width, height = map(float, values)
        x_center = x_left + width / 2
        y_center = y_top + height / 2
        box_centers_file.append((x_center, y_center))
        # print(f"Center of the box: ({x_center}, {y_center})")

# Compare to the box centers obtained from funcget_tracked_data
print("\nBox centers from funcget_tracked_data:")
box_centers_func = list(zip(xy_track_top[0][:10], xy_track_top[1][:10]))
# for center in box_centers_func:
    # print(f"Center of the box: {center}")

# Compare the two lists
print("\nComparison of box centers:")
for i, (file_center, func_center) in enumerate(zip(box_centers_file, box_centers_func)):
    print(f"Box {i + 1}: File center = {file_center}, Funcget_tracked_data center = {func_center}")

cv2.destroyAllWindows()

Number of top images: 10
Number of side images: 10
Number of top x and y coordinates: 10, 10
Number of side x and z coordinates: 10, 10
Length of xz_track_side: 82, 82
Length of xy_track_top: 82, 82
Event frame range for top: 10 to 175
Event frame range for side: 6 to 171
Length of sliced x_coor_top: 10
Length of sliced y_coor_top: 10
Length of sliced x_coor_side: 10
Length of sliced z_coor_side: 10
fr=10
Box centers from the file:

Box centers from funcget_tracked_data:

Comparison of box centers:
Box 1: File center = (1688.0, 1618.0), Funcget_tracked_data center = (1688.0, 1618.0)
Box 2: File center = (1688.0, 1618.0), Funcget_tracked_data center = (1688.0, 1618.0)
Box 3: File center = (1689.0, 1620.0), Funcget_tracked_data center = (1689.0, 1620.0)
Box 4: File center = (1695.5, 1619.0), Funcget_tracked_data center = (1695.5, 1619.0)
Box 5: File center = (1701.0, 1615.5), Funcget_tracked_data center = (1701.0, 1615.5)
Box 6: File center = (1706.0, 1615.5), Funcget_tracked_data center

In [13]:
# Create a video from top-side-front images
# Each list inside images corresponds to a different view (e.g., top, side, front)
exp_num = 9
exp_path = rf"C:\Users\Amir\Documents\PHD\Experiments\Force Measurements\Exp2h_tilt\{exp_num}"
top_path = os.path.join(exp_path, "top1_cropped\Croped_1")
top_images = [os.path.join(top_path, f) for f in os.listdir(top_path) if f.endswith('.JPG')][0:750:10]
side_path = os.path.join(exp_path, "side1_cropped\Croped_1")
side_images = [os.path.join(side_path, f) for f in os.listdir(side_path) if f.endswith('.JPG')][:750:10]
front_path = os.path.join(exp_path, "front1_cropped\Croped_1")
front_images = [os.path.join(front_path, f) for f in os.listdir(front_path) if f.endswith('.JPG') ][:750:10]
print(fr"Number of top images: {len(top_images)}")
images = [top_images, side_images, front_images]
# Create a video with the images
output_path = os.path.join(exp_path, "output", "parallel_video.mp4")
parallel_video2(images, fps=5, fixed_height=400, save_video=True, output_path=output_path)


Number of top images: 75
Final frame displayed. Close the window to end.


In [17]:
# Create a video from images in a folder
import cv2
import os

def create_video_from_images(image_folder, output_video_path, fps=30, frame_size=None,file_total=100, skip=1):
    """
    Create a video from images in a folder.

    Parameters:
        image_folder (str): Path to the folder containing images.
        output_video_path (str): Full path to save the output video (including .mp4).
        fps (int): Frames per second for the video.
        frame_size (tuple): Desired frame size (width, height). If None, uses first image size.
    """
    # Get a sorted list of image file paths
    image_files = sorted([
        os.path.join(image_folder, f) 
        for f in os.listdir(image_folder) 
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])[:file_total:skip]

    if not image_files:
        raise ValueError(f"No valid images found in folder: {image_folder}")
    
    # Load the first image to determine size if needed
    first_image = cv2.imread(image_files[0])
    if first_image is None:
        raise ValueError(f"Failed to read the first image: {image_files[0]}")

    if frame_size is None:
        frame_size = (first_image.shape[1], first_image.shape[0])  # width, height

    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is None:
            print(f"Warning: Unable to read image {image_file}. Skipping.")
            continue

        if (img.shape[1], img.shape[0]) != frame_size:
            img = cv2.resize(img, frame_size)

        video_writer.write(img)

    video_writer.release()
    print(f"Video saved at: {output_video_path}")

# Example usage
image_folder = r"C:\Users\Amir\Documents\PHD\Experiments\Force Measurements\Exp2h_tilt\4\side1_cropped\Croped_1"
output_video_path = r"C:\Users\Amir\Documents\PHD\Experiments\Force Measurements\Exp2h_tilt\4\side1_cropped\output_video.mp4"
create_video_from_images(image_folder, output_video_path, fps=5,file_total=500,skip=10)

Video saved at: C:\Users\Amir\Documents\PHD\Experiments\Force Measurements\Exp2h_tilt\4\side1_cropped\output_video.mp4
